# Capítulo 01 - Introdução ao Apache Iceberg

## 📚 O que é Apache Iceberg?

Apache Iceberg é um **formato de tabela aberto** para grandes datasets analíticos.

### 🎯 Principais Características:

**1. Schema Evolution** - Adicione, remova ou renomeie colunas sem reescrever dados

**2. Partition Evolution** - Mude o particionamento sem reprocessar dados

**3. Time Travel** - Consulte versões anteriores dos dados

**4. ACID Transactions** - Garantias transacionais completas

**5. Performance** - Predicate pushdown eficiente

Neste notebook vamos:
1. Criar uma tabela Iceberg com PyIceberg
2. Ler dados com DuckDB
3. Explorar a estrutura de metadados

## 1. Setup e Imports

In [ ]:
import duckdb
import os
import shutil
import pandas as pd
from pyiceberg.catalog import load_catalog
import pyarrow as pa

print(f"DuckDB: {duckdb.__version__}")
print(f"Pandas: {pd.__version__}")
print(f"PyArrow: {pa.__version__}")
print("\n✓ Imports completos!")

In [ ]:
def safe_install_ext(con, ext_name):
    """Instala e carrega extensão DuckDB."""
    try:
        con.install_extension(ext_name)
        con.load_extension(ext_name)
        print(f"✓ Extension '{ext_name}' carregada")
        return True
    except Exception as e:
        print(f"✗ Falha: {e}")
        return False

print("✓ Helper definida")

In [ ]:
print("🚀 Capítulo 01: Apache Iceberg")
print("="*50)

## 2. Criar Warehouse

In [ ]:
WAREHOUSE_PATH = "./iceberg_warehouse"

if os.path.exists(WAREHOUSE_PATH):
    shutil.rmtree(WAREHOUSE_PATH)
    print("✓ Diretório removido")

os.makedirs(WAREHOUSE_PATH, exist_ok=True)
print(f"✓ Warehouse: {os.path.abspath(WAREHOUSE_PATH)}")

## 3. Criar Catalog

In [ ]:
print("📦 Criando Catalog...")

catalog = load_catalog(
    "local", 
    **{
        "type": "sql", 
        "uri": f"sqlite:///{WAREHOUSE_PATH}/catalog.db",
        "warehouse": f"file://{os.path.abspath(WAREHOUSE_PATH)}"
    }
)

print("✓ Catalog criado (SQL/SQLite)")

## 4. Criar Namespace

In [ ]:
print("📁 Criando namespace...")

try:
    catalog.create_namespace("default")
    print("✓ Namespace 'default' criado")
except:
    print("⚠️  Namespace já existe")

print(f"Namespaces: {catalog.list_namespaces()}")

## 5. Criar Tabela

In [ ]:
print("📊 Preparando dados...")

df_pandas = pd.DataFrame({
    'id': [1, 2, 3],
    'name': ['A', 'B', 'C']
})

print(df_pandas)

df_arrow = pa.Table.from_pandas(df_pandas)
print(f"\n✓ Arrow Table criada")
print(f"Schema: {df_arrow.schema}")

In [ ]:
print("🔨 Criando tabela...")

# Dropar tabela se existe
try:
    catalog.drop_table("default.my_table")
    print("✓ Tabela antiga removida")
except:
    pass

table = catalog.create_table(
    "default.my_table",
    schema=df_arrow.schema,
)

print(f"✓ Tabela: default.my_table")
print(f"Nome: {table.name()}")
print(f"Schema: {table.schema()}")

In [ ]:
print("💾 Inserindo dados...")
print("⚠️  Nota: PyIceberg append() tem incompatibilidade com PyArrow 15.0.0")
print("⚠️  Erro: store_decimal_as_integer não suportado")

try:
    table.append(df_arrow)

    print("✓ Dados inseridos!")
    print(f"\nLocation: {table.location}")

except Exception as e:

    print(f"✗ Erro ao inserir: {str(e)[:100]}...")    
    print("\n💡 Solução: Downgrade PyArrow ou use pyiceberg.io directly")

## 6. Explorar Estrutura

In [ ]:
tables = catalog.list_tables("default")
print(f"Tabelas: {tables}")

In [ ]:
print("🔍 Procurando metadata...")

search_path = os.path.join(WAREHOUSE_PATH, "default", "my_table", "metadata")
metadata_file = None

if os.path.exists(search_path):
    files = [f for f in os.listdir(search_path) if f.endswith(".metadata.json")]
    if files:
        files.sort()
        metadata_file = os.path.join(search_path, files[-1])
        print(f"✓ Metadata: {metadata_file}")
    else:
        print("✗ Nenhum metadata.json")
else:
    print(f"✗ Diretório não existe")

## 7. Ler com DuckDB

In [ ]:
print("🦆 Configurando DuckDB...")

con = duckdb.connect()
iceberg_loaded = safe_install_ext(con, "iceberg")
safe_install_ext(con, "parquet")

In [ ]:
print("📖 Lendo com DuckDB...")

if iceberg_loaded and metadata_file:
    try:
        result = con.execute(f"SELECT * FROM iceberg_scan('{metadata_file}')").df()
        print("✓ Query executada!")
        print("\nResultado:")
        print(result)
    except Exception as e:
        print(f"✗ Erro: {e}")
else:
    print("✗ Iceberg ou metadata não disponível")

## 8. Estrutura de Diretórios

In [ ]:
print("📁 Estrutura do Warehouse:")
print("="*50)

for root, dirs, files in os.walk(WAREHOUSE_PATH):
    level = root.replace(WAREHOUSE_PATH, '').count(os.sep)
    indent = ' ' * 2 * level
    print(f"{indent}{os.path.basename(root)}/")
    subindent = ' ' * 2 * (level + 1)
    for file in files[:10]:  # Limitar a 10 arquivos
        print(f"{subindent}{file}")

## ✅ Resumo

**Aprendemos:**
1. Apache Iceberg - formato de tabela avançado
2. PyIceberg - criar tabelas
3. DuckDB - ler com `iceberg_scan()`
4. Estrutura do warehouse

**Próximo:** Capítulo 02 - Instalação e Configuração

In [ ]:
print("\n" + "="*50)
print("✅ CAPÍTULO 01 CONCLUÍDO!")
print("="*50)

# Capítulo 01: Introdução ao Apache Iceberg

Este notebook apresenta conceitos fundamentais do Apache Iceberg integrado com DuckDB.

**Objetivo:** Entender o que é Iceberg e realizar análises básicas em tabelas Iceberg.

**Conteúdo:**
- Instalação e configuração inicial
- Conceitos básicos de Iceberg
- Primeira query em tabela Iceberg

## 3. Primeira Query em Tabela Iceberg

Vamos criar uma conexão e fazer uma query exemplo (nota: o caminho da tabela deve ser ajustado para seu ambiente).

## 2. O que é Apache Iceberg?

**Apache Iceberg** é um formato de tabela aberto para dados analíticos de larga escala que oferece:

- **Time Travel**: Consultar dados históricos
- **Schema Evolution**: Modificar schema sem reescrever dados
- **ACID Transactions**: Garantias transacionais
- **Particionamento Eficiente**: Otimização automática de queries

**DuckDB + Iceberg** = Análise SQL rápida em tabelas Iceberg, sem precisar de Spark!

In [ ]:
# Importar bibliotecas e função auxiliar
import duckdb
import importlib.util


def has_module(name):
    """Verifica se um módulo Python está disponível"""
    return importlib.util.find_spec(name) is not None


def safe_install_ext(con, ext_name):
    """Instala e carrega extensão DuckDB com tratamento de erros"""
    try:
        con.execute(f"INSTALL {ext_name}")
        con.execute(f"LOAD {ext_name}")
        print(f"✅ Extensão '{ext_name}' instalada e carregada com sucesso")
        return True
    except Exception as e:
        print(f"⚠️  Aviso: Falha ao instalar/carregar {ext_name}: {e}")
        return False


print("Funções auxiliares carregadas!")

In [ ]:
# Instalar DuckDB se necessário
import sys
!{sys.executable} -m pip install -q duckdb pyarrow

## 1. Instalação e Configuração Inicial

Primeiro, vamos instalar as dependências necessárias e carregar a extensão Iceberg.

In [ ]:
# Criar conexão e instalar extensão Iceberg
con = duckdb.connect()
safe_install_ext(con, "iceberg")

# Exemplo de query em tabela Iceberg
# NOTA: Ajuste o caminho para uma tabela Iceberg real em seu ambiente
try:
    result = con.execute("""
        SELECT category, sum(revenue) as total_revenue
        FROM iceberg_scan('s3://bucket/sales')
        GROUP BY category
        LIMIT 10
    """).df()
    
    print("✅ Query executada com sucesso!")
    print(result)
except Exception as e:
    print(f"ℹ️  Exemplo demonstrativo - ajuste o caminho da tabela Iceberg")
    print(f"Erro: {e}")

## 4. Próximos Passos

Neste capítulo você aprendeu:
- ✅ Como instalar e configurar DuckDB com Iceberg
- ✅ Conceitos básicos do Apache Iceberg
- ✅ Estrutura básica de queries em tabelas Iceberg

**Próximo:** Capítulo 02 - Instalação e Configuração detalhada